In [7]:
import pandas as pd
from pyserini.search.lucene import LuceneSearcher

# Membaca Dataframe

In [8]:
df = pd.read_csv('../artikel/all_health_articles.csv')
df.drop(columns=['description', 'content'], inplace=True)
df.head()

,article_index,title,tag,link,date,source
0,1,5 rawat ginjal sehat cegah sakit ginjal,Kesehatan Umum,https://www.biofarma.co.id/id/announcement/det...,3 September 2025,Biofarma
1,2,7 tips jaga sehat mata,Kesehatan Umum,https://www.biofarma.co.id/id/announcement/det...,29 Agustus 2025,Biofarma
2,3,sepele 10 sebab campak abai,Kesehatan Umum,https://www.biofarma.co.id/id/announcement/det...,28 Agustus 2025,Biofarma
3,4,campak sebab gejala cegah,Kesehatan Umum,https://www.biofarma.co.id/id/announcement/det...,25 Agustus 2025,Biofarma
4,5,mudah jaga sehat gigi mulut hindar dari sakit,Kesehatan Umum,https://www.biofarma.co.id/id/announcement/det...,20 Agustus 2025,Biofarma


In [16]:
# Fungsi untuk print hasil
def display_results(query, hits):
    print(f'\n{query}:')

    if not hits:
        print("Tidak ada dokumen yang sesuai.")
        return
    
    # Create a list to store results with scores
    results_with_scores = []
    
    print('Results:')
    for i in range(len(hits)):
        # Get the corresponding row from dataframe
        matching_row = df[df['article_index'] == int(hits[i].docid)].copy()
        
        if not matching_row.empty:
            # Add score to the dataframe row
            matching_row = matching_row.copy()
            matching_row['score'] = hits[i].score
            matching_row['rank'] = i + 1
            results_with_scores.append(matching_row)
            
            # print(f'{i+1:2} ID: {hits[i].docid:4} Score: {hits[i].score:.5f}')
            # print(f'   Title: {matching_row.iloc[0]["title"]}')
            # print()
    
    # Combine all results into a single DataFrame
    if results_with_scores:
        result_df = pd.concat(results_with_scores, ignore_index=True)
        return result_df
    else:
        print("No matching documents found in DataFrame.")
        return pd.DataFrame()

# Bm25 Retrieval (Title)

In [9]:
searcher = LuceneSearcher('../indexing/indexes/title_jsonl')

queries = [
  'gejala diabetes',
  'makanan sehat untuk jantung',
  'cara mencegah hipertensi',
  'efek samping vaksin covid-19',
  'manfaat olahraga yoga'
]

In [13]:
# Test the function with a query
query = queries[0]  # 'gejala diabetes'
hits = searcher.search(query)  # Get top 5 results

# Display results and get DataFrame with scores
result_df = display_results(query, hits)

# You can now work with the result DataFrame
print("\nResult DataFrame with scores:")
result_df


gejala diabetes:
Results:

Result DataFrame with scores:


,article_index,title,tag,link,date,source,score,rank
0,86,gejala diabetes melitus tahu,Kesehatan Umum,https://www.biofarma.co.id/id/announcement/det...,11 Desember 2024,Biofarma,3.715400,1
1,410,gejala diabetes tipe 1 anak minum berat badan ...,HEALTH,https://health.kompas.com/read/25I12181500168/...,12 September 2025,kompas,3.203300,2
2,49,5 faktor risiko diabetes tahu,Kesehatan Umum,https://www.biofarma.co.id/id/announcement/det...,25 Maret 2025,Biofarma,2.141600,3
3,51,bahaya diabetes mellitus faktor risiko tingkat,Kesehatan Umum,https://www.biofarma.co.id/id/announcement/det...,25 Maret 2025,Biofarma,2.077200,4
4,201,hari diabetes dunia diabetasol gelar edukasi s...,Berita Kesehatan,https://www.klikdokter.com/info-sehat/berita-k...,Oktober November 2022,Klikdokter,2.016600,5
5,367,7 camil enak idap diabetes alpukat almond,Kesehatan Umum,https://health.detik.com/berita-detikhealth/d-...,15 September 2025,detik,2.016599,6
6,414,diabetes tipe 1 anak diagnosis cegah mati,HEALTH,https://health.kompas.com/read/25I11191500768/...,11 September 2025,kompas,2.016598,7
7,353,efek samping metformin guna diabetes tipe 2 ti...,kesehatan,https://www.alodokter.com/efek-samping-metform...,11 September 2025,alodokter,1.905400,8
8,103,stres gejala sebab cegah,Kesehatan Umum,https://www.biofarma.co.id/id/announcement/det...,21 Oktober 2024,Biofarma,1.505400,9
9,4,campak sebab gejala cegah,Kesehatan Umum,https://www.biofarma.co.id/id/announcement/det...,25 Agustus 2025,Biofarma,1.505399,10


# Bm25 Retrieval (Title + Content)

In [14]:
searcher = LuceneSearcher('../indexing/indexes/title_content_jsonl')

queries = [
  'gejala diabetes',
  'makanan sehat untuk jantung',
  'cara mencegah hipertensi',
  'efek samping vaksin covid-19',
  'manfaat olahraga yoga'
]

In [17]:
# Test the function with a query
query = queries[0]  # 'gejala diabetes'
hits = searcher.search(query)  # Get top 5 results

# Display results and get DataFrame with scores
result_df = display_results(query, hits)

# You can now work with the result DataFrame
print("\nResult DataFrame with scores:")
result_df


gejala diabetes:
Results:

Result DataFrame with scores:


,article_index,title,tag,link,date,source,score,rank
0,86,gejala diabetes melitus tahu,Kesehatan Umum,https://www.biofarma.co.id/id/announcement/det...,11 Desember 2024,Biofarma,2.2208,1
1,49,5 faktor risiko diabetes tahu,Kesehatan Umum,https://www.biofarma.co.id/id/announcement/det...,25 Maret 2025,Biofarma,2.1344,2
2,410,gejala diabetes tipe 1 anak minum berat badan ...,HEALTH,https://health.kompas.com/read/25I12181500168/...,12 September 2025,kompas,1.9226,3
3,353,efek samping metformin guna diabetes tipe 2 ti...,kesehatan,https://www.alodokter.com/efek-samping-metform...,11 September 2025,alodokter,1.8921,4
4,201,hari diabetes dunia diabetasol gelar edukasi s...,Berita Kesehatan,https://www.klikdokter.com/info-sehat/berita-k...,Oktober November 2022,Klikdokter,1.8563,5
5,342,5 efek samping minum rebus daun ciplukan waspada,hidup sehat,https://www.alodokter.com/5-efek-samping-minum...,10 September 2025,alodokter,1.8166,6
6,51,bahaya diabetes mellitus faktor risiko tingkat,Kesehatan Umum,https://www.biofarma.co.id/id/announcement/det...,25 Maret 2025,Biofarma,1.7918,7
7,317,6 efek samping methylprednisolone,kesehatan,https://www.alodokter.com/6-efek-samping-methy...,11 September 2025,alodokter,1.7372,8
8,64,kapulaga rempah ajaib gudang manfaat sehat,Kesehatan Umum,https://www.biofarma.co.id/id/announcement/det...,21 Februari 2025,Biofarma,1.6883,9
9,1,5 rawat ginjal sehat cegah sakit ginjal,Kesehatan Umum,https://www.biofarma.co.id/id/announcement/det...,3 September 2025,Biofarma,1.6611,10


# Bm25 Retrieval (Title + Desc + Content)

In [20]:
searcher = LuceneSearcher('../indexing/indexes/title_desc_content_jsonl')

queries = [
  'gejala diabetes',
  'makanan sehat untuk jantung',
  'cara mencegah hipertensi',
  'efek samping vaksin covid-19',
  'manfaat olahraga yoga'
]

In [21]:
# Test the function with a query
query = queries[0]  # 'gejala diabetes'
hits = searcher.search(query)  # Get top 5 results

# Display results and get DataFrame with scores
result_df = display_results(query, hits)

# You can now work with the result DataFrame
print("\nResult DataFrame with scores:")
result_df


gejala diabetes:
Results:

Result DataFrame with scores:


,article_index,title,tag,link,date,source,score,rank
0,86,gejala diabetes melitus tahu,Kesehatan Umum,https://www.biofarma.co.id/id/announcement/det...,11 Desember 2024,Biofarma,2.2057,1
1,49,5 faktor risiko diabetes tahu,Kesehatan Umum,https://www.biofarma.co.id/id/announcement/det...,25 Maret 2025,Biofarma,2.1629,2
2,410,gejala diabetes tipe 1 anak minum berat badan ...,HEALTH,https://health.kompas.com/read/25I12181500168/...,12 September 2025,kompas,2.0167,3
3,353,efek samping metformin guna diabetes tipe 2 ti...,kesehatan,https://www.alodokter.com/efek-samping-metform...,11 September 2025,alodokter,1.9762,4
4,201,hari diabetes dunia diabetasol gelar edukasi s...,Berita Kesehatan,https://www.klikdokter.com/info-sehat/berita-k...,Oktober November 2022,Klikdokter,1.8422,5
5,414,diabetes tipe 1 anak diagnosis cegah mati,HEALTH,https://health.kompas.com/read/25I11191500768/...,11 September 2025,kompas,1.8149,6
6,51,bahaya diabetes mellitus faktor risiko tingkat,Kesehatan Umum,https://www.biofarma.co.id/id/announcement/det...,25 Maret 2025,Biofarma,1.8014,7
7,342,5 efek samping minum rebus daun ciplukan waspada,hidup sehat,https://www.alodokter.com/5-efek-samping-minum...,10 September 2025,alodokter,1.7947,8
8,317,6 efek samping methylprednisolone,kesehatan,https://www.alodokter.com/6-efek-samping-methy...,11 September 2025,alodokter,1.7118,9
9,64,kapulaga rempah ajaib gudang manfaat sehat,Kesehatan Umum,https://www.biofarma.co.id/id/announcement/det...,21 Februari 2025,Biofarma,1.6805,10
